In [26]:
# Import modules
from sqlalchemy import create_engine, MetaData, Table, update
from sqlalchemy import Column, Integer, String, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.engine.url import URL

import arxiv
import pandas as pd
from datetime import datetime

In [27]:
# Used full file path for testing purpose
json_file = pd.read_json("/Users/EmilyWang/Downloads/ops-master/paper-collector/DeepLearningArticles.json", orient='index')
json_file = json_file.head(20)

In [28]:
# Extracting information
json_file['published_date'] = json_file['published'].str.extract('(\d\d\d\d-\d\d-\d\d)', expand=True)
json_file['published_time'] = json_file['published'].str.extract('(\d\d:\d\d:\d\d)', expand=True)
json_file['updated_date'] = json_file['updated'].str.extract('(\d\d\d\d-\d\d-\d\d)', expand=True)
json_file['updated_time'] = json_file['updated'].str.extract('(\d\d:\d\d:\d\d)', expand=True)
json_file['unique_id'] = json_file['id'].str.extract('(\d\d\d\d\.\d\d\d\d\d)', expand=True)
json_file['version_number'] = json_file['id'].str.extract('(\d$)', expand=True)

final_json = json_file[['unique_id', 'version_number', 'author', 
                        'title', 'summary', 'arxiv_comment', 
                        'published_date', 'published_time', 
                        'updated_date', 'updated_time', 
                        'tags', 'authors']]

final_json = final_json.drop_duplicates(subset='unique_id', keep='first', inplace=False)

In [29]:
def obtain_new_articles():
    new_articles = arxiv.query(search_query, max_results=5000,
                               sort_by="lastUpdatedDate", sort_order="descending")
    new_articles_df = pd.DataFrame.from_dict(new_articles)
    ordered_new_articles = new_articles_df.reindex(
                            columns=['title', 'author', 'authors', 'id', 'arxiv_comment',
                                     'arxiv_primary_category', 'published', 'summary',
                                     'tags', 'updated'])
    return ordered_new_articles

def extract_column(df_file):
    # Extracting information
    df_file['published_date'] = df_file['published'].str.extract('(\d\d\d\d-\d\d-\d\d)', expand=True)
    df_file['published_time'] = df_file['published'].str.extract('(\d\d:\d\d:\d\d)', expand=True)
    df_file['updated_date'] = df_file['updated'].str.extract('(\d\d\d\d-\d\d-\d\d)', expand=True)
    df_file['updated_time'] = df_file['updated'].str.extract('(\d\d:\d\d:\d\d)', expand=True)
    df_file['unique_id'] = df_file['id'].str.extract('(\d\d\d\d\.\d\d\d\d\d)', expand=True)
    df_file['version_number'] = df_file['id'].str.extract('(\d$)', expand=True)

    final_df = df_file[['unique_id', 'version_number', 'author', 
                        'title', 'summary', 'arxiv_comment', 
                        'published_date', 'published_time', 
                        'updated_date', 'updated_time', 
                        'tags', 'authors']]

    final_df = final_df.drop_duplicates(subset='unique_id', keep='first', inplace=False)
    return final_df

def check_existence(session, PaperTable, id_string):
    query = session.query(PaperTable).filter(PaperTable.id==id_string)
    
    if query.one_or_none():
        return True
    else:
        return False

def update_existing_articles(session, PaperTable, id_string, df):
    now = datetime.now
    update(PaperTable).where(PaperTable.id==id_string).\
            values(version = int(df.iloc[0, 1]), \
                   summary = df.iloc[0, 4], \
                   arxiv_comment = df.iloc[0, 5], \
                   updated_date = df.iloc[0, 8], \
                   updated_time = df.iloc[0, 9])

    session.commit()

def extract_category(df):
    tags = df.iloc[0, 10]
    tags_list = []
    for i in range(len(tags)):
        tags_list.append([tags[i]['term']])
    return tags_list

def insert_new_articles(session, PaperTable, AuthorTable, TagTable, id_string, df):
    # Adding records into Paper Table
    paper_row = PaperTable(id = id_string, 
                           version = int(df.iloc[0, 1]), 
                           author = df.iloc[0, 2], 
                           title = df.iloc[0, 3], 
                           summary = df.iloc[0, 4], 
                           arxiv_comment = df.iloc[0, 5], 
                           published_date = df.iloc[0, 6], 
                           published_time = df.iloc[0, 7], 
                           updated_date = df.iloc[0, 8], 
                           updated_time = df.iloc[0, 9])
    session.add(paper_row)
    session.commit()
    
    # Adding records into Author Table
    authors = df.iloc[0, 11]
    for i in range(len(authors)):
        id_str = id_string + "-" + str(i)
        author_row = AuthorTable(id = id_str, 
                                 author = authors[i], 
                                 author_entry = paper_row)
        session.add(author_row)
    session.commit()
    
    # Adding records into Tag Table
    tags_list = extract_category(df)
    for i in range(len(tags_list)):
        id_str = id_string + "-" + str(i)
        tag_row = TagTable(id = id_str, 
                           paper_tag = tags_list[i], 
                           tag_entry = paper_row)
        session.add(tag_row)
    session.commit()

In [5]:
# Local db url
db_url = {'drivername': 'postgres',
          'username': 'postgres',
          'password': 'postgres',
          'host': '127.0.0.1',
          'port': 5432}
engine = create_engine(URL(**db_url))

# Initiate base
Base = declarative_base()

# Create tables
class PaperTable(Base):
    __tablename__ = 'PaperTable'
    id = Column(String, primary_key=True)
    version = Column(Integer, nullable=False)
    author = Column(String, nullable=False)
    authors = relationship('AuthorTable', 
                           backref='author_entry')
    tags = relationship('TagTable', 
                        backref='tag_entry')
    title = Column(String, nullable=False)
    summary = Column(String)
    arxiv_comment = Column(String)
    published_date = Column(String)
    published_time = Column(String)
    updated_date = Column(String)
    updated_time = Column(String)
    timestamp = Column(DateTime, default=datetime.utcnow)

class AuthorTable(Base):
    __tablename__ = 'AuthorTable'
    id = Column(String, primary_key=True)
    author = Column(String, nullable=False)
    paper_id = Column(String, ForeignKey('PaperTable.id'))

class TagTable(Base):
    __tablename__ = 'TagTable'
    id = Column(String, primary_key=True)
    paper_tag = Column(String, nullable=False)
    paper_id = Column(String, ForeignKey('PaperTable.id'))

Base.metadata.create_all(bind=engine)

Session = sessionmaker(bind=engine)
session = Session()

In [30]:
final_json_lst = final_json['unique_id'].values

for id_string in final_json_lst:
    row_df = final_json[final_json['unique_id']==id_string]

    if check_existence(session, PaperTable, id_string):
        exist_version = session.query(PaperTable.version).filter(PaperTable.id==id_string)
        if int(final_json[final_json['unique_id']==id_string]['version_number']) \
                > exist_version[0]:
            update_existing_articles(session, PaperTable, id_string, row_df)

    else:
        insert_new_articles(session, PaperTable, AuthorTable, TagTable, id_string, row_df)

In [40]:
from sqlalchemy import and_
# Testing if foreign keys/relationships work
query = session.query(PaperTable, AuthorTable, TagTable).filter(and_(PaperTable.id==AuthorTable.paper_id, 
                                                                PaperTable.id==TagTable.paper_id)).limit(20)
for _p, _a, _t in query.all():
    print(_p.id, _a.author, _t.paper_tag)

1803.08607 Mickey Aleksic {cs.CV}
1803.08607 Liang Shen {cs.CV}
1803.08607 Xiaopeng Zhang {cs.CV}
1803.08607 Shaojie Zhuo {cs.CV}
1803.08607 Chen Feng {cs.CV}
1803.08607 Tao Sheng {cs.CV}
1903.05071 Pavlos Protopapas {cs.NE}
1903.05071 Nikos Gianniotis {cs.NE}
1903.05071 Jacob Reinier Maat {cs.NE}
1903.05071 Pavlos Protopapas {cs.LG}
1903.05071 Nikos Gianniotis {cs.LG}
1903.05071 Jacob Reinier Maat {cs.LG}
1903.05071 Pavlos Protopapas {stat.ML}
1903.05071 Nikos Gianniotis {stat.ML}
1903.05071 Jacob Reinier Maat {stat.ML}
1903.05063 Daniel Wintz {cs.LG}
1903.05063 Elliott Wolf {cs.LG}
1903.05063 Michael Lingzhi Li {cs.LG}
1903.05063 Daniel Wintz {stat.ML}
1903.05063 Elliott Wolf {stat.ML}


In [ ]:
### Complete
# Import modules
from sqlalchemy import create_engine, MetaData, Table, update
from sqlalchemy import Column, Integer, String, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.engine.url import URL

import arxiv
import pandas as pd
from datetime import datetime

# Use create_tables when first setting up the database
def create_tables():
    # Initiate base
    Base = declarative_base()
    
    # Create tables
    class PaperTable(Base):
        __tablename__ = 'PaperTable'
        id = Column(String, primary_key=True)
        version = Column(Integer, nullable=False)
        author = Column(String, nullable=False)
        authors = relationship('AuthorTable', 
                               backref='author_entry')
        tags = relationship('TagTable', 
                            backref='tag_entry')
        title = Column(String, nullable=False)
        summary = Column(String)
        arxiv_comment = Column(String)
        published_date = Column(String)
        published_time = Column(String)
        updated_date = Column(String)
        updated_time = Column(String)
        timestamp = Column(DateTime, default=datetime.utcnow)
        
    class AuthorTable(Base):
        __tablename__ = 'AuthorTable'
        id = Column(String, primary_key=True)
        author = Column(String, nullable=False)
        paper_id = Column(String, ForeignKey('PaperTable.id'))
        
    class TagTable(Base):
        __tablename__ = 'TagTable'
        id = Column(String, primary_key=True)
        paper_tag = Column(String, nullable=False)
        paper_id = Column(String, ForeignKey('PaperTable.id'))
        
    Base.metadata.create_all(bind=engine)

# max_results set to 1000
def obtain_new_articles():
    new_articles = arxiv.query(search_query, max_results=1000,
                               sort_by="lastUpdatedDate", 
                               sort_order="descending")
    new_articles_df = pd.DataFrame.from_dict(new_articles)
    ordered_new_articles = new_articles_df.reindex(
                            columns=['title', 'author', 'authors', 'id', 'arxiv_comment',
                                     'arxiv_primary_category', 'published', 'summary',
                                     'tags', 'updated'])
    return ordered_new_articles

def extract_column(df_file):
    # Extracting information
    df_file['published_date'] = df_file['published'].str.extract('(\d\d\d\d-\d\d-\d\d)', expand=True)
    df_file['published_time'] = df_file['published'].str.extract('(\d\d:\d\d:\d\d)', expand=True)
    df_file['updated_date'] = df_file['updated'].str.extract('(\d\d\d\d-\d\d-\d\d)', expand=True)
    df_file['updated_time'] = df_file['updated'].str.extract('(\d\d:\d\d:\d\d)', expand=True)
    df_file['unique_id'] = df_file['id'].str.extract('(\d\d\d\d\.\d\d\d\d\d)', expand=True)
    df_file['version_number'] = df_file['id'].str.extract('(\d$)', expand=True)

    final_df = df_file[['unique_id', 'version_number', 'author', 
                        'title', 'summary', 'arxiv_comment', 
                        'published_date', 'published_time', 
                        'updated_date', 'updated_time', 
                        'tags', 'authors']]

    final_df = final_df.drop_duplicates(subset='unique_id', 
                                        keep='first', inplace=False)
    return final_df

def check_existence(session, PaperTable, id_string):
    query = session.query(PaperTable).filter(PaperTable.id==id_string)
    
    if query.one_or_none():
        return True
    else:
        return False

def update_existing_articles(session, PaperTable, id_string, df):
    now = datetime.now
    update(PaperTable).where(PaperTable.id==id_string).\
            values(version = int(df.iloc[0, 1]), \
                   summary = df.iloc[0, 4], \
                   arxiv_comment = df.iloc[0, 5], \
                   updated_date = df.iloc[0, 8], \
                   updated_time = df.iloc[0, 9])

    session.commit()

def extract_category(df):
    tags = df.iloc[0, 10]
    tags_list = []
    for i in range(len(tags)):
        tags_list.append([tags[i]['term']])
    return tags_list

def insert_new_articles(session, PaperTable, AuthorTable, TagTable, 
                        id_string, df):
    # Adding records into Paper Table
    paper_row = PaperTable(id = id_string, 
                           version = int(df.iloc[0, 1]), 
                           author = df.iloc[0, 2], 
                           title = df.iloc[0, 3], 
                           summary = df.iloc[0, 4], 
                           arxiv_comment = df.iloc[0, 5], 
                           published_date = df.iloc[0, 6], 
                           published_time = df.iloc[0, 7], 
                           updated_date = df.iloc[0, 8], 
                           updated_time = df.iloc[0, 9])
    session.add(paper_row)
    session.commit()
    
    # Adding records into Author Table
    authors = df.iloc[0, 11]
    for i in range(len(authors)):
        id_str = id_string + "-" + str(i)
        author_row = AuthorTable(id = id_str, 
                                 author = authors[i], 
                                 author_entry = paper_row)
        session.add(author_row)
    session.commit()
    
    # Adding records into Tag Table
    tags_list = extract_category(df)
    for i in range(len(tags_list)):
        id_str = id_string + "-" + str(i)
        tag_row = TagTable(id = id_str, 
                           paper_tag = tags_list[i], 
                           tag_entry = paper_row)
        session.add(tag_row)
    session.commit()


def main():
    # Local db url
    db_url = {'drivername': 'postgres',
              'username': 'postgres',
              'password': 'postgres',
              'host': '127.0.0.1',
              'port': 5432}
    engine = create_engine(URL(**db_url))

    # reflect the tables
    metadata = MetaData()
    Base = automap_base(metadata=metadata)
    Base.prepare(engine, reflect=True)

    # Mapped classes with names matching that of the table name
    PaperTable = Base.classes.PaperTable
    AuthorTable = Base.classes.AuthorTable
    TagTable = Base.classes.TagTable

    Session = sessionmaker(bind=engine)
    session = Session()
    
    # article_df = extract_column(obtain_new_articles())
    article_df = extract_column(pd.read_json("/Users/EmilyWang/Desktop/ops/paper-collector/DeepLearningArticles.json", 
                                             orient='index'))
    article_id_lst = article_df['unique_id'].values

    for id_string in article_id_lst:
        row_df = article_df[article_df['unique_id']==id_string]
        
        if check_existence(session, PaperTable, id_string):
            exist_version = session.query(PaperTable.version).\
                                    filter(PaperTable.id==id_string).one()
            if int(article_df.iloc[0, 1]) > exist_version[0]:
                update_existing_articles(session, 
                                         PaperTable, id_string, row_df)
                
        else:
            insert_new_articles(session, PaperTable, AuthorTable, 
                                TagTable, id_string, row_df)


main()


In [ ]:
# Discarded
def load_session():
    # Local db url
    db_url = {'drivername': 'postgres',
              'username': 'postgres',
              'password': 'postgres',
              'host': '127.0.0.1',
              'port': 5432}

    # Initiate engine
    engine = create_engine(URL(**db_url))
    metadata = MetaData()
    metadata.reflect(bind=engine)
    PaperTable = metadata.tables['PaperTable']
    AuthorTable = metadata.tables['AuthorTable']
    TagTable = metadata.tables['TagTable']
    
    # Create session
    Session = sessionmaker(bind=engine)
    session = Session()
    return session, PaperTable, AuthorTable, TagTable